En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

In [3]:
file_path = "farmers-protest-tweets-2021-2-4.json"

In [4]:
%load_ext line_profiler
%load_ext memory_profiler

# Pregunta 1

Se tomaron dos enfoques distintos. El primero consistió en leer los datos con la librería JSON y luego trabajar la información con Pandas. Para el segundo enfoque se utilizó PySpark para la lectura y tratamiento de los datos.

En cuanto al tiempo de ejecución con el primer método, se observó que los tiempos eran estables y repetibles, con una duración de alrededor de 11 segundos. En cambio, el segundo método dependía de forma sustancial del estado de la sesión de Spark. En la primera ejecución consecutiva, los tiempos de ejecución rondaban los 15 segundos, muy distintos a los tiempos de las ejecuciones posteriores, que rondaban entre 5 y 6 segundos.

Debido a esto, se tomó la decisión de elegir el primer método para q1_time, únicamente por su estabilidad. Cabe considerar que, en el caso de tener un ambiente de ejecución más estable, el segundo método podría ofrecer mejores resultados a largo plazo.

En cuanto al uso de memoria, este fue un análisis más directo. Con el método 1, siempre se observó un uso de hasta 1300 MiB, muy por encima del método 2, que rondaba los 300 MiB de forma constante. Por ende, se seleccionó el método 2 para q1_memory

## Metodo 1

Se ejecuto multiples veces y siempre se consigui un tiempo de ejecucion de alrededor de 11s

In [8]:
from q1_time import  q1_time

In [ ]:
%lprun -f q1_time q1_time(file_path=file_path)

In [ ]:
%lprun -f q1_time q1_time(file_path=file_path)

In [ ]:
%mprun -f q1_time q1_time(file_path=file_path)

Mem pregunta 1

In [3]:
from q1_memory import  q1_memory

In [ ]:
%lprun -f q1_memory q1_memory(file_path=file_path)

In [ ]:
%lprun -f q1_memory q1_memory(file_path=file_path)

In [ ]:
%mprun -f q1_memory q1_memory(file_path=file_path)

# Pregunta 2

Para este problema se tomo un aproach similar que la pregunta anterior, pero a diferencia de esta, para el tiempo se tomo pyspark y para la memoria funiones convencionales.
Dado que solo se tienen que contar ocurrencias a nivel registro, se puede ir iterando fila a fila. 

Lamentablemente este aproach no obtuvo los resultado esperados y la metodologia sin pyspark utilizo practicamente la misma memoria que la con, y mas en algunas pruebas.

In [5]:
from q2_time import  q2_time

%lprun -f q2_time q2_time(file_path=file_path)

your 131072x1 screen size is bogus. expect trouble
24/10/14 22:54:59 WARN Utils: Your hostname, pescara-pc resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/10/14 22:54:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/14 22:55:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Timer unit: 1e-09 s

Total time: 18.4042 s
File: /home/sandbox/files/latam-challenge/src/q2_time.py
Function: q2_time at line 44

Line #      Hits         Time  Per Hit   % Time  Line Contents
    44                                           def q2_time(file_path: str) -> List[Tuple[str, int]]:
    45         1 3905414685.0    4e+09     21.2      spark = SparkSession.builder.appName(APP_NAME).getOrCreate()
    46                                           
    47         1 5956659988.0    6e+09     32.4      data = spark.read.json(file_path)
    48         1  335665521.0    3e+08      1.8      dataframe = get_most_used_emojis(spark, data)
    49                                           
    50         1       1793.0   1793.0      0.0      del data
    51                                           
    52         2      27091.0  13545.5      0.0      output = [
    53         1 7216616653.0    7e+09     39.2          tuple(row) for row in dataframe.select([col("emoji"), col("count")]).co

In [6]:
%lprun -f q2_time q2_time(file_path=file_path)

Timer unit: 1e-09 s

Total time: 8.03239 s
File: /home/sandbox/files/latam-challenge/src/q2_time.py
Function: q2_time at line 44

Line #      Hits         Time  Per Hit   % Time  Line Contents
    44                                           def q2_time(file_path: str) -> List[Tuple[str, int]]:
    45         1   97870683.0    1e+08      1.2      spark = SparkSession.builder.appName(APP_NAME).getOrCreate()
    46                                           
    47         1  810689662.0    8e+08     10.1      data = spark.read.json(file_path)
    48         1  111067536.0    1e+08      1.4      dataframe = get_most_used_emojis(spark, data)
    49                                           
    50         1        581.0    581.0      0.0      del data
    51                                           
    52         2       9938.0   4969.0      0.0      output = [
    53         1 6016603940.0    6e+09     74.9          tuple(row) for row in dataframe.select([col("emoji"), col("count")]).co

In [7]:
%mprun -f q2_time q2_time(file_path=file_path)

Filename: /home/sandbox/files/latam-challenge/src/q2_time.py

Line #    Mem usage    Increment  Occurrences   Line Contents
    44    174.9 MiB    174.9 MiB           1   def q2_time(file_path: str) -> List[Tuple[str, int]]:
    45    174.9 MiB      0.0 MiB           1       spark = SparkSession.builder.appName(APP_NAME).getOrCreate()
    46                                         
    47    174.9 MiB      0.0 MiB           1       data = spark.read.json(file_path)
    48    174.9 MiB      0.0 MiB           1       dataframe = get_most_used_emojis(spark, data)
    49                                         
    50    174.9 MiB      0.0 MiB           1       del data
    51                                         
    52    174.9 MiB      0.0 MiB          24       output = [
    53    174.9 MiB      0.0 MiB          11           tuple(row) for row in dataframe.select([col("emoji"), col("count")]).collect()
    54                                             ]
    55                      

In [8]:
from q2_memory import  q2_memory

%lprun -f q2_memory q2_memory(file_path)

*** KeyboardInterrupt exception caught in code being profiled.

Timer unit: 1e-09 s

Total time: 9.83701 s
File: /home/sandbox/files/latam-challenge/src/q2_memory.py
Function: q2_memory at line 7

Line #      Hits         Time  Per Hit   % Time  Line Contents
     7                                           def q2_memory(file_path: str) -> List[Tuple[str, int]]:
     8         1        912.0    912.0      0.0      result = dict()
     9         2     451684.0 225842.0      0.0      with open(file_path, "r") as f:
    10                                                   # En el archivo hay un json por linea, hay que itearr y extrar cada uno
    11      3080   55925861.0  18157.7      0.6          for line in f:
    12      3080 8193672422.0    3e+06     83.3              data = pd.json_normalize(json.loads(line))
    13      6158  311252272.0  50544.4      3.2              for c in data.content.to_list():
    14      4139 1274725765.0 307979.2     13.0                  for e in emoji.emoji_list(c):
    15      1060     983372.0    927.7      0.0    

In [ ]:
%mprun -f q2_memory q2_memory(file_path)